In [8]:
%pip install langchain openai langchain_community langchain_openai huggingface_hub langchain_core

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
lst_cls = ['Политика',
 'Экономика',
 'Общество',
 'Закон и право',
 'Кино',
 'Телевидение',
 'Персоны',
 'События',
 'Бренды',
 'Наука',
 'Гаджеты',
 'Соцсети',
 'Технологии',
 'Опросы',
 'Головоломки',
 'Дом',
 'Транспорт',
 'Погода',
 'Рецепты',
 'Мода',
 'Красота',
 'Дети',
 'Авто',
 'Питомцы',
 'Здоровье',
 'спорт',
 'туризм',
 'забота о себе']

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI

# from tqdm.auto import tqdm


base_url = "http://mistral.vkcloud.eazify.net:8000/v1"
# base_url = "http://hackllm.vkcloud.eazify.net:8000/v1"
# base_url = "http://shwars.vkcloud.eazify.net:8000/v1"

chat = ChatOpenAI(api_key="<key>",
                model = "tgi",
                openai_api_base = base_url,
                temperature=0.1)

parser = JsonOutputParser()
# text = 'исполняющая обязанности президента боливии жанин аньес рассказала что в ближайшие несколько дней будет выдан ордер на арест беглого эксглавы государства эво моралеса она заявила об этом местной газете los tiempos в частности моралеса решили арестовать за неуважение конституции боливии и ее переписывание по мнению аньес бывший президент должен ответить перед правосудием она рассказала что ходатайства уже поданы и добавила что действия моралеса должны расценивать как подстрекательство к мятежу и терроризму аньес также заявила что боливия нуждается в переменах она призвала моралеса соблюдать статус политического эмигранта и оставить страну в покое по ее словам новой власти удалось достигнуть примирения и необходимо провести выборы эво моралес объявил об отставке 10 ноября на фоне продолжающихся протестов после чего укрылся в мексике вместе с другими представителями руководства страны при этом он назвал произошедшее госпереворотом и заявил что готов вернуться в страну массовые беспорядки в боливии начались после объявления результатов выборов главы государства которые прошли 20 октября на них победу в первом туре одержал моралес его соперник экспрезидент карлос меса не признал итоги голосования власти обвинили в подтасовке результатов россия признала аньес временным президентом боливии в москве назвали события которые привели к отставке моралеса эквивалентными госперевороту однако посчитали происходящее внутренним делом страны'

instruct = """
На основе предоставленного текста определи одну наиболее подходящую категорию и выдай ему вероятность пренадлежности (от 0 до 1, где чем ближе к 1, тем выше вероятность). Затем извлеки ключевые слова для этой категории. 
Оформи ответ в формате JSON: {{
            "selected_label": "категория из списка",
            "prob": 0.8,
            "hashtags": ["хэштэг1", "хэштэг2", ...], 
            "keywords": ["ключевое слово1", "ключевое слово2", ...]
            }}

Предоставленный текст: ```{text}```
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", f"Ты сторого следуешь задаче: твоя задача извлекать из текста ключевые слова, выдавать все подходящие хэштэги, основываясь на список категорий, и выдавать его категорию только из следующего списка и ему вероятность пренадлежности (от 0 до 1, где чем ближе к 1, тем выше вероятность): {lst_cls}. Придумывать свою категорию нельзя"),
        ("human", instruct),
    ]
)


# res = chat.invoke(chat_template.format_messages(text=text))
# res = parser.invoke(res)
# print(res)
# print(res.content)

In [15]:
import pandas as pd

df = pd.read_csv("./shared/krasota.csv")

In [18]:
len(df)

2299

In [16]:
lst = [str(x).strip() for x in df['Article Text'].tolist() if x]

In [29]:
krasota_json = []

for i, txt in enumerate(lst):
    try:
        res = parser.invoke(chat.invoke(chat_template.format_messages(text=txt)))
        krasota_json.append(res)
        print(i, res)
    except Exception as e:
        print(e)

0 {'selected_label': 'Здоровье', 'prob': 0.9, 'hashtags': ['#Здоровье', '#Косметика', '#Уход', '#Скрабы', '#Мицеллярная вода', '#Масло', '#Термальная вода', '#Ночной крем'], 'keywords': ['кожа', 'уход', 'косметика', 'скрабы', 'м']}
1 {'selected_label': 'Здоровье', 'prob': 0.9, 'hashtags': ['#Здоровье', '#Ресницы', '#Уход'], 'keywords': ['ресницы', 'уход', 'питание', 'масла', 'компрессы', 'лечебные препараты', 'массаж', 'витамины']}
2 {'selected_label': 'Здоровье', 'prob': 0.8, 'hashtags': ['брови', 'волосы', 'здоровье', 'питание', 'маски', 'масла', 'витамины'], 'keywords': ['густые брови', 'рассказ', 'отрастить', 'питание', 'маски', 'масла', 'витами']}
3 {'selected_label': 'Кино', 'prob': 0.9, 'hashtags': ['#маникюр', '#юбилейный', '#лайм', '#маникюр', '#юбилейный', '#бьюти', '#кино'], 'keywords': ['маникюр', 'юбилейный', 'лайм', 'бьюти', 'кино']}
4 {'selected_label': 'Здоровье', 'prob': 0.9, 'hashtags': ['здоровье', 'кожа', 'волосы', 'душ', 'маникюр'], 'keywords': ['кожа', 'волосы', '

In [14]:
!ls shared/

300k_7topics		   links_golovolomki.txt   opros.csv
340k_10topics		   links_health.txt	   persona_1.csv
355k_13topics		   links_kino.txt	   persona_2.csv
50k_25cats_FINAL.pkl	   links_opros_RT_853.txt  persony_MAIL_V2.txt
BERT-BASE-VKHACK.zip	   links_pets.txt	   persony_lenta.txt
MLM_50k_25cats_FINAL.pkl   links_pravo_RG_ru.txt   persony_mail.txt
auto.csv		   links_science.txt	   pets.csv
classifier_training.ipynb  links_socnets.txt	   pogoda_mail.txt
data-preparation.ipynb	   links_technology.txt    pravo_RG_ru.csv
gadgets.csv		   links_tv.txt		   recipe.csv
krasota.csv		   mlruns		   socnets.csv
links_auto.txt		   moda.csv		   tv.csv
links_gadgets.txt	   model2		   zagadki.csv
